In [ ]:
from bs4 import BeautifulSoup as bs
import requests as r
import lxml
import json
import pprint
import re
import sqlite3
import time
from datetime import datetime
import random
import argparse
import math
import sys
import re

In [ ]:
# --- 共用模組 ---
import public_crawler
# --- 信義房屋專用模組 ---
import crawler_sy

#### 查看DB的結構

In [5]:
with sqlite3.connect('sy.db') as conn:
    c = conn.cursor()
    pprint.pprint(list(c.execute("SELECT name FROM sqlite_master WHERE type='table';")))
conn.close()

[('rental',)]


In [6]:
# with sqlite3.connect('sy.db') as conn:
#     c = conn.cursor()
#     qry = "Create Table rental (url text unique not null,  title text, main text, detail text,  nosql text,  rdb text,  getTime text )"
#     c.execute(qry)
# conn.close()

In [7]:
# with sqlite3.connect('sy.db') as conn:
#     c = conn.cursor()
#     c.execute('delete from rental')
#     c.execute('select count(*) from rental')
#     print(c.fetchall())
# conn.close()

In [8]:
# with sqlite3.connect('sy.db') as conn:
#     c = conn.cursor()
#     qry = "select count(*) from rental"
#     c.execute(qry)
#     print(c.fetchall())
# conn.close()

In [9]:
# 1.外頁網址, 依照不同房仲網, 解析出各自的urls
def extractAllUrls(dbName, soup):
    try:
        allUrls = []
        if dbName == 'sy.db':
            for url_raw in soup.select('a.name'):
                url = url_raw['href']
                allUrls.append(url)
            return allUrls
        elif dbName == 'yc.db':
            pass
        elif dbName == 'jc.db':
            pass
        elif dbName == 'hf.db':
            pass
        else:
            print('沒有這個db...' + dbName + 'public_crawler.extractAllUrls')
 
    except Exception as err:
        print('解析allUrls Error' + dbName)
#

In [10]:
# 3.查詢內頁全部 _ 逐項拆解
def queryByURL(url):
    try:
        res = r.get(url)
        soup = bs(res.text, 'lxml')
        
        # 內頁標題
        title = extract_title_sy(soup, url)
        # 內頁表格主要資訊
        main = extract_main_sy(soup, url)
        # 內頁表格明細資訊
        detail = extract_detail_sy(soup, url)
        
        return title, main, detail
        
    except Exception as err:
        print(title + " ; 內頁soup.select時Error ; " + url)
#

In [11]:
# 抓外頁網址
# 給我
# 1. 要儲存的dbName
# 2. outerPageETL()回傳的urls
# 這支程式會做1件事情: 叫另外2支程式做事情
# 1. 爬取每個網址內的頁面資訊 -> soupText (str datatype)
# 2. 把soupText依照dbName存到所屬的sqlite     ""public_crawler.py""
# !!!!! 睡 1 秒 !!!!!
def innerPageETL(dbName, urls):
    i = 0
    for url in urls:
        try:
            i = i + innerPageETL_one(dbName, url)
        except Exception as err:
            print('innerPageETL --- Something Error')
        time.sleep(1)
    return i

# 同innerPageETL, 但僅執行一筆
def innerPageETL_one(dbName, url):
    try:
        # 信義房屋會檢查headers
        title, main, detail = queryByURL(url)

        if (title == '' or isinstance(main, str) or isinstance(detail, str)):
            return 0
        
        else:
            return insertToDB(dbName, url, title, str(main), str(detail))
#             return public_crawler.insertToDB(dbName, url, title, str(main), str(detail))
        
    except Exception as err:
        print(str(err) + ' ; url = ' + url)

In [12]:
# 儲存內頁到所屬資料庫
# 告訴我你要放進去的
# Sqlite Name,  url,  soup
# 會幫你把 url , soup 依照現在時間, 存入你指定的database的rental table內
# 回傳1 表示成功匯入
# 回傳0 表示之前已經存過此網址
# 回傳0 + print(xxx ; Error url : xxx)  表示資料庫連接有問題
#  -> 可能是你把不適合的資料塞進來了 or
#  -> create table本身就有點問題
def insertToDB(dbName, url, title, main, detail):
    try:
        with sqlite3.connect(dbName) as conn:
            c = conn.cursor()
            # 執行查詢字串
            getTime = str(datetime.now())
            queryString = "SELECT url FROM rental WHERE url = '" + url + "'"
            c.execute(queryString)
            if (len(c.fetchall()) == 0):
                insertStr = 'INSERT INTO rental(url, title, main, detail, getTime) VALUES (?, ?, ?, ?, ?)'
                values = (url, title, main, detail, getTime)
                
                c.execute(insertStr, values)
                return 1
                
            else:
                # 重複出現的網址(不儲存)
                print(dbName + ' : ' + url + ' ---------------- repeat')
                return 0
                
    except Exception as err:
        print(str(err) + ' ; Error url : ' + url)
        return 0
    finally:
        conn.close()

In [13]:
# 外網解析時, 抓到網址清單, 逐筆檢查此url是否已有
def saveThisUrl(dbName, url):
    try:
        with sqlite3.connect(dbName) as conn:
            c = conn.cursor()
            c.execute("SELECT url FROM rental WHERE url = '" + url + "'")
            if (len(c.fetchall()) == 0):
                return True
            else:
                return False

    except Exception as err:
        print('Repeat check error with(不明錯誤) : ' + url)

In [14]:
# 特定頁面所有網址, 到資料庫看有沒有儲存過, 沒有的話再把他堆疊, 最後回傳
def filterUrls(dbName, all_urls):
    urls = []
    for url in all_urls:
        if saveThisUrl(dbName, url):
            urls.append(url)
            
    return urls

### 外頁總頁數

In [40]:
def getTotalPage_sy(url):
    res = r.get(url)
    soup = bs(res.text, 'lxml')
    nums = soup.select('h1')[0].text

    regex = re.compile('\d*[,]?\d+')
    nums = int(regex.search(nums).group().replace(',', ''))
    return math.ceil(int(nums) / 20)

In [41]:
# 衣外網抓總頁數
src = "http://rent.sinyi.com.tw/AJAX/ajax_searchItem.php?search=1&b=26"
getTotalPage_sy(src)

2

### 外頁爬取

In [12]:
url = 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=CA17405'
res = r.get(url)
soup = bs(res.text, 'lxml')

In [10]:
# 取得網址清單
tmp = soup.select('a.name')
urls = []
for url in tmp:
    urls.append(url['href'])
urls

['http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=237747',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=237383',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236742',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236738',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236697',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236696',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236691',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236687',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236479',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236476',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236457',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236452',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236296',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236295',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236292',
 'http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=236284',
 'http:/

### 內頁明細

In [6]:
### 內頁拆解
def extract_title(soup, url):
    try:
        title = soup.select('h1')[0].text
        
    except Exception as err:
        print('title error : ' + url)
        title = ''
    finally:
        return title
#


# 主要段落
def extract_main(soup, url):
    main = {}
    try:
        soup_main = soup.select('div#summary')[0]    
        main['addr'] = soup_main.select('div.addr')[0].text #大概在哪邊
        main['rent'] = soup_main.select('span.rent')[0].text    # 租金
        if soup_main.select('span.deposite'):
            main['deposite'] = soup_main.select('span.deposite')[0].text    # 押金
        main['type'] = soup_main.select('div.type')[0].text.replace('房屋類型', '')    # 房屋類型 
        main['size'] = soup_main.select('td')[0].text    # 坪數
        main['floor'] = soup_main.select('td')[1].text    # 樓層
        main['pattern'] = soup_main.select('td')[2].text    # 格局

        #生活周邊
        env = '' 
        env_list = soup.find_all('a',{'href':'javascript:;'}) 
        for i in range(len(env_list) -1 ): #最後個是加入追蹤不加入迴圈,故減去一
            env = env + env_list[i]['title'] + "、" #每加一次後面加上個 (、)  方便辨識
        main['envir'] = env
        
        #經緯度
        if soup.find('img',id='static_map2') != None:
            lat_and_lng = soup.find('img',id='static_map2')['src'].replace('.png','').split('_')
            main['latlng'] = lat_and_lng[-2] + ',' + lat_and_lng[-1]
        else:
            main['latlng'] = ''
        
    except Exception as err:
        print('main error : ' + url)
        main = ''
        
    finally:
        return main
#


# 詳細資料
def extract_detail(soup, url):
    detail = {}
    try:
        detail_keys = soup.find('section',id='mainInfo').find_all('th')
        detail_values = soup.find('section',id='mainInfo').find_all('td')
        detail = {key.text.replace('\u3000','') : values.text.replace(' ','').replace('\t','').replace('\n','') 
                       for key , values in zip(detail_keys, detail_values)} 

        #食、住、行、樂 如果是車位/土地/場地,  則無 周邊生活資訊的tag
        house_type = soup.find('div','type').text.replace('房屋類型','')
        if house_type != '土地/場地' and house_type != '車位':
            live_demand = soup.find('section',id='surround').find_all('p')
            for i in live_demand:
                detail[i.find('span').text] = i.text[1:]

        if soup.select("#agreement"):
            for ttr in soup.select('#agreement')[0].select('tr'):
                detail[ttr.select('th')[0].text] = ttr.select('td')[0].text.replace('\t', '').replace('\n', '')
        
    except Exception as err:
        print('detail error : ' + url)
        detail = ''
        
    finally:
        return detail
#

In [16]:
lat_and_lng = soup.find('img',id='static_map2')['src'].replace('.png','').split('_')

lat_and_lng[-2] + ',' + lat_and_lng[-1]

'25.0536,121.6009'

In [14]:
if soup.select('span.deposite'):    # 押金
    print('a')

### 內頁解析 - 測試程式

In [18]:
url = "http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=CA15248"
res = r.get(url)
soup = bs(res.text, 'lxml')

In [19]:
soup.select('input#GPSLatiude')

[]

In [20]:
soup.select('input#GPSLongitude')

[]

In [3]:
float(soup.select('input#GPSLongitude')[0]['value'])

IndexError: list index out of range

In [22]:
main

{'addr': '天母揚明\u3000\u3000\u3000\u3000\u3000\u3000(CA15248)',
 'deposite': '押金：110000元',
 'envir': '',
 'floor': '2樓/ 共7樓',
 'latlng': '-0.0022,118.7618',
 'pattern': '3房2廳2衛',
 'rent': '55,000元',
 'size': '43.53坪',
 'type': '整層住宅'}

In [21]:
title = extract_title(soup, url)
main = extract_main(soup, url)
detail = extract_detail(soup, url)

In [13]:
detail_keys = soup.find('section',id='mainInfo').find_all('th')
detail_values = soup.find('section',id='mainInfo').find_all('td')
detail = {key.text.replace('\u3000','') : values.text.replace(' ','').replace('\t','').replace('\n','') 
               for key , values in zip(detail_keys, detail_values)} 

#食、住、行、樂 如果是車位/土地/場地,  則無 周邊生活資訊的tag
house_type = soup.find('div','type').text.replace('房屋類型','')

if house_type != '土地/場地' and house_type != '車位':
    live_demand = soup.find('section',id='surround').find_all('p')
    for i in live_demand:
        detail[i.find('span').text] = i.text[1:]

if soup.select("#agreement"):

    for ttr in soup.select('#agreement')[0].select('tr'):
        print(ttr)
        mm[ttr.select('th')[0].text] = ttr.select('td')[0].text.replace('\t', '').replace('\n', '')
        
mm

NameError: name 'mm' is not defined

In [19]:
catch_details(url)

{'details': {'傢俱設備': '否',
  '出租坪數': '166.64坪',
  '地址': '台北市內湖區新湖一路',
  '屋齡': '332000元',
  '建物坪數': '196.7坪',
  '押金': '17,000元',
  '暗房': '是',
  '格局': '4房2廳',
  '樓層': '4樓/總樓層7',
  '租金': '166,000元/月',
  '管理警衛': '',
  '管理費': '本物件係為分隔出租，出租面積約為166.64坪。然實際大小應以現場丈量、目測為準。',
  '車位': '5年',
  '邊間': '無',
  '類型': '辦公/廠房',
  '食': '(市場、商圈) 近傳統市場'},
 'getTime': '2017-06-23',
 'main': {'建物坪數': '196.7坪',
  '房屋類型': '辦公/廠房',
  '押金': '332000元',
  '格局': '4房2廳',
  '樓層': '4樓/ 共7樓',
  '生活周邊': '',
  '租金': '\n166,000元/月租\n                                                        押金：332000元\n',
  '經緯度': '25.0586,121.5759'},
 'title': '瑞士百坪帶裝潢'}

In [18]:
def catch_details(url,json_signal=False):
    
    from bs4 import BeautifulSoup
    from datetime import date
    import requests
    import json
    import lxml
    
    res = requests.get(url)
    info = BeautifulSoup(res.text,'lxml')
    all_dict = {}
    
    #-----------------------------------去除區(土地、車位)-----------------------------------------#
    
    house_type = info.find('div','type').text.replace('房屋類型','')
    if house_type == '土地/場地' or house_type == '車位':
        pass
    
    else:

        #-----------------------------------標題、getTime---------------------------------------------#

        #標題
        all_dict['title'] = info.find('h1').text
        #更新時間
        all_dict['getTime'] = str(date.today())



        #-----------------------------------主要資訊區---------------------------------------------#

        main_info = {}

        #租金
        main_info['租金'] = info.find('div','price').text
        #押金
        if info.find('span','deposite') != None :
            main_info['押金'] = info.find('span','deposite').text.replace('押金：','')
        else:
            main_info['押金'] = '' #如果要簡化資料庫效率，可在這轉換為數字0，

        #房屋類型
        if info.find('div','type') != None :
            main_info['房屋類型'] = info.find('div','type').text.replace('房屋類型','')
        else:
            main_info['房屋類型'] =''

        #建物坪數、樓層、格局
        main_keys = info.find('table').find_all('th')
        main_values = info.find('table').find_all('td')
        main_items = {key.text : values.text for key , values in zip(main_keys,main_values)} #迴圈進入dict
        main_info.update(main_items) #dict合併

        #生活周邊
        env = '' 
        env_list = info.find_all('a',{'href':'javascript:;'}) 
        for i in range(len(env_list) -1 ): #最後個是加入追蹤不加入迴圈,故減去一
            env = env + env_list[i]['title'] + "、" #每加一次後面加上個 (、)  方便辨識
        main_info['生活周邊'] = env

        #經緯度
        if info.find('img',id='static_map2') != None:
            lat_and_lng = info.find('img',id='static_map2')['src'].replace('.png','').split('_')
            main_info['經緯度'] = lat_and_lng[-2] + ',' + lat_and_lng[-1]
        else:
            main_info['經緯度'] = ''

        all_dict['main'] = main_info

        #-----------------------------------詳細資料區-----------------------------------------------#
        #地址、租金、類型、格局、樓層、遷入日期、建物坪數、管理費、押金、朝向、屋齡、車位、邊間、暗房、隔間材料、傢俱設備、
        #其他室友、房東同住、房東身份、仲介服務費、最短租期、辦理租約公證、房客提供身份證件備存、管理警衛(有些有有些無)
        details_info = {}
        details_keys = info.find('section',id='mainInfo').find_all('th')
        details_values = info.find('section',id='mainInfo').find_all('td')
        details_info = {key.text.replace('\u3000','') : values.text.replace(' ','').replace('\t','').replace('\n','') 
                       for key , values in zip(details_keys,details_values)} 

        #食、住、行、樂
        live_demand = info.find('section',id='surround').find_all('p')
        for i in live_demand:
            details_info[i.find('span').text] = i.text[1:]

        all_dict['details'] = details_info


        if json_signal == False:
            return all_dict
        else:
            return json.dumps(all_dict)

In [1]:
for r in soup.select('#mainInfo')[0]:
    pass
#     print(r)
#     print('---------------')

NameError: name 'soup' is not defined

In [32]:
with sqlite3.connect('jc.db') as conn:
    c = conn.cursor()
    c.execute('drop table rental')
c.close()
conn.close()